In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img

base_dir = 'Dataset'
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   validation_split=0.2
                                   )
validation_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [ ]:
image_size = (224, 224)
train_data_dir = '/content/drive/MyDrive/images/train'
test_data_dir = '/content/drive/MyDrive/images/test'
training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 seed=101,                                                 
                                                 target_size=image_size,
                                                 batch_size=32,
                                                 subset = "training",
                                                 class_mode='categorical')

Found 4603 images belonging to 3 classes.


In [ ]:
validation_set = validation_gen.flow_from_directory(test_data_dir, 
                                               target_size=image_size,
                                               batch_size=32, 
                                               subset = "validation",
                                               class_mode='categorical')

Found 286 images belonging to 3 classes.


In [ ]:
## Conv2D model

import keras
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,LSTM, Reshape
from tensorflow.keras.layers import Activation, Dense, Dropout
model = Sequential([
            
    Conv2D(filters=32,kernel_size=(3,3),  input_shape = (224, 224, 3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    # LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.25),
    Dense(3, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 54, 54, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 64)       

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy(),precision_m,recall_m,f1_m])

In [ ]:
# for Conv2D model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callback = EarlyStopping(monitor='loss', patience=3)
history = model.fit(training_set,validation_data=validation_set, epochs=30,callbacks=[callback])

Epoch 1/30
144/144 [==============================] - 1914s 13s/step - loss: 1.1611 - categorical_accuracy: 0.3441 - precision_m: 0.0167 - recall_m: 0.0106 - f1_m: 0.0118 - val_loss: 1.0981 - val_categorical_accuracy: 0.3462 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/30
144/144 [==============================] - 69s 482ms/step - loss: 1.0983 - categorical_accuracy: 0.3374 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00 - val_loss: 1.0980 - val_categorical_accuracy: 0.3427 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/30
144/144 [==============================] - 69s 480ms/step - loss: 1.0982 - categorical_accuracy: 0.3289 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00 - val_loss: 1.0977 - val_categorical_accuracy: 0.3427 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/30
144/144 [==============================] - 69s 481

In [ ]:
#for Conv2D model

metrics_train = model.evaluate(training_set)
metrics_test = model.evaluate(validation_set)

print(f"TRAIN_SET: {metrics_train}")
print("--------------------------------------------")
print(f"TEST_SET: {metrics_test}")

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save("/content/drive/MyDrive/models/CNN2D.h5")